<a href="https://colab.research.google.com/github/Ranjan-Kashyap/UK-Cars-FineTuneLLM/blob/main/UK_Car_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Add Kaggle Dataset to Google Collab**

In [2]:
pip install kaggle


In [3]:
mkdir ~/.kaggle


In [ ]:
!pwd


/content


In [ ]:
!ls /

bin			    etc     media		      root  tools
boot			    home    mnt			      run   usr
content			    lib     NGC-DL-CONTAINER-LICENSE  sbin  var
cuda-keyring_1.0-1_all.deb  lib32   opt			      srv
datalab			    lib64   proc		      sys
dev			    libx32  python-apt		      tmp


In [4]:
cp /kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d adityadesai13/used-car-dataset-ford-and-mercedes


  0% 0.00/1.10M [00:00<?, ?B/s]
100% 1.10M/1.10M [00:00<00:00, 153MB/s]


In [6]:
mkdir data

In [7]:
!unzip used-car-dataset-ford-and-mercedes.zip -d data

Archive:  used-car-dataset-ford-and-mercedes.zip
  inflating: data/audi.csv           
  inflating: data/bmw.csv            
  inflating: data/cclass.csv         
  inflating: data/focus.csv          
  inflating: data/ford.csv           
  inflating: data/hyundi.csv         
  inflating: data/merc.csv           
  inflating: data/skoda.csv          
  inflating: data/toyota.csv         
  inflating: data/unclean cclass.csv  
  inflating: data/unclean focus.csv  
  inflating: data/vauxhall.csv       
  inflating: data/vw.csv             


In [2]:
!pip install transformers
!pip install torchtext


In [3]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments


In [4]:
df = pd.read_csv('data/audi.csv')

In [5]:
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0


In [6]:
# Assume df is your DataFrame
def row_to_text(row):
    return f"Model: {row['model']}, Year: {row['year']}, Price: {row['price']}, Transmission: {row['transmission']}, Mileage: {row['mileage']}, Fuel Type: {row['fuelType']}, Tax: {row['tax']}, MPG: {row['mpg']}, Engine Size: {row['engineSize']}"

df['text'] = df.apply(row_to_text, axis=1)
df['label'] = 0  # Dummy label, as we're not doing classification


In [51]:
from sklearn.model_selection import train_test_split

# Divide into train and test for the 50% model
train_df, test_df = train_test_split(df, test_size=0.5, random_state=42)

# For training on 50% of the data (which is actually 25% of the original dataset)
train_df_50, _ = train_test_split(train_df, test_size=0.5, random_state=42)

# For training on 100% of the data
train_df_100 = df.copy()

# Separate test set for 100% training (Optional: you can take another subset or use other strategies)
test_df_100 = df.sample(frac=0.1, random_state=42)  # 10% of the data as a test set for example

# Now check the shapes
print("Shape of train_df:", train_df.shape)
print("Shape of test_df:", test_df.shape)
print("Shape of train_df_50:", train_df_50.shape)
print("Shape of train_df_100:", train_df_100.shape)
print("Shape of test_df_100:", test_df_100.shape)


Shape of train_df: (5334, 11)
Shape of test_df: (5334, 11)
Shape of train_df_50: (2667, 11)
Shape of train_df_100: (10668, 11)
Shape of test_df_100: (1067, 11)


In [53]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_batch(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_data_50 = train_df_50[['text', 'label']]
train_data_100 = train_df_100[['text', 'label']]

train_data_50 = train_data_50.rename(columns={'text':'sentence1'})
train_data_100 = train_data_100.rename(columns={'text':'sentence1'})

# Convert DataFrames to dictionaries with list values
train_data_50_dict = train_data_50.to_dict(orient='list')
train_data_100_dict = train_data_100.to_dict(orient='list')

# Tokenize using only the 'sentence1' values (which are in list format)
train_encodings_50 = tokenizer(train_data_50_dict['sentence1'], padding=True, truncation=True)
train_encodings_100 = tokenizer(train_data_100_dict['sentence1'], padding=True, truncation=True)



**Fine tuning and Hyperparameter Tuning**

In [54]:
model_50 = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=1)
model_100 = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=1)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Define training and arguments to set hyperparameters**

In [10]:
pip install accelerate -U

In [11]:
pip install transformers[torch]

In [12]:
!pip install transformers[torch]


In [13]:
!pip install accelerate -U


In [55]:
training_args_50 = TrainingArguments(
    output_dir='./results_50',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_50',
)

training_args_100 = TrainingArguments(
    output_dir='./results_100',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_100',
)

trainer_50 = Trainer(
    model=model_50,
    args=training_args_50,
    train_dataset=train_data_50,
)

trainer_100 = Trainer(
    model=model_100,
    args=training_args_100,
    train_dataset=train_data_100,
)


In [58]:
train_data_50.head()

,sentence1,label
6699,"Model: A5, Year: 2017, Price: 22500, Transmis...",0
4743,"Model: Q2, Year: 2017, Price: 23490, Transmis...",0
8533,"Model: A5, Year: 2019, Price: 28950, Transmis...",0
6293,"Model: A7, Year: 2016, Price: 20200, Transmis...",0
10654,"Model: A3, Year: 2016, Price: 16495, Transmis...",0


In [39]:
n = 1286
row_n = train_data_50.iloc[n]

print("Row at index", n, "is:")
print(row_n)

Row at index 1286 is:
sentence1    Model:  A5, Year: 2019, Price: 30728, Transmis...
label                                                        0
Name: 546, dtype: object


In [56]:
trainer_50.train()
trainer_100.train()


KeyError: ignored